In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import sys

sys.path.append(os.path.abspath("../"))
from dataset import DS_NAMES
from dataset import DS_DIMENSIONS
from utilities.figure_tools import get_latest_results_directory, make_fig_directory, load_results

results_path = "../results/turing/vary_nrects.csv"
output_dir = "./vary_nrects/"
export_figures = True

if export_figures and not os.path.isdir(output_dir):
    os.makedirs(output_dir)


In [2]:
all_results = pd.read_csv(results_path).groupby(["dataset", "explainer", "n_rects"]).mean().reset_index()
print("datasets", list(all_results["dataset"].unique()))
all_results.head()

datasets ['cancer', 'glass', 'magic', 'spambase', 'vertebral']


,dataset,explainer,n_rects,n_trees,max_depth,iteration,accuracy,precision,recall,f1,per_valid,avg_dist,avg_length,prep_time,explain_time,sample_time,n_explain
0,cancer,FACETIndex,100,100.0,5.0,2.0,0.947368,0.948096,0.901243,0.924017,1.0,0.528811,16.82,0.207964,0.061618,0.003081,20.0
1,cancer,FACETIndex,200,100.0,5.0,2.0,0.947368,0.948096,0.901243,0.924017,1.0,0.499250,16.20,0.376845,0.077780,0.003889,20.0
2,cancer,FACETIndex,300,100.0,5.0,2.0,0.947368,0.948096,0.901243,0.924017,1.0,0.485451,16.15,0.542867,0.094966,0.004748,20.0
3,cancer,FACETIndex,400,100.0,5.0,2.0,0.947368,0.948096,0.901243,0.924017,1.0,0.467345,15.78,0.711589,0.113338,0.005667,20.0
4,cancer,FACETIndex,500,100.0,5.0,2.0,0.947368,0.948096,0.901243,0.924017,1.0,0.451967,15.62,0.885681,0.129520,0.006476,20.0


In [3]:
# drop some values for visual clarity
all_results = all_results[all_results["n_rects"].isin([200, 300, 400, 500, 600, 700, 800, 900]) == False]

In [4]:
cmap = plt.get_cmap("tab10")
colors = cmap.colors
ds_colors = {
    "cancer": colors[0],
    "glass": colors[1],
    "magic": colors[2],
    "spambase": colors[3],
    "vertebral": colors[4]
}
ds_markers = {
    "cancer": "^",
    "glass": "o",
    "magic": "v",
    "spambase": "s",
    "vertebral": "D"
}
marker_size = 8
line_width = 2
marker_sizes = {
    "cancer": marker_size,
    "glass": marker_size,
    "magic": marker_size,
    "spambase": marker_size,
    "vertebral": marker_size
}

In [5]:
def render_plot(xfeature, yfeature, xlabel, ylabel, title, save_name=None, xrange=None, yrange=None, xlog=False, ylog=False, lloc=None, loc="best", lhorz=False):

    fig = plt.figure(figsize=[6.4, 4.8], frameon=True)
    ax = fig.add_subplot(1, 1, 1)
    for ds in all_results["dataset"].unique():
        is_ds = all_results["dataset"] == ds
        for expl in all_results["explainer"].unique():
            is_expl = all_results["explainer"] == expl
            matching_rows = is_ds & is_expl
            ax.plot(all_results[matching_rows][xfeature], all_results[matching_rows][yfeature], label=ds, lw=line_width, marker=ds_markers[ds], ms=marker_sizes[ds], color=ds_colors[ds])

        # title and axis labels
        if title is not None:
            plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        # legend
        if lhorz:
            plt.legend(loc=loc, ncol=len(all_results["dataset"].unique()), bbox_to_anchor=(0, 1.09, 1, 0), handletextpad=0.5, columnspacing=1, handlelength=1.5)
        else:
            plt.legend(loc=loc)
        # x and y ranges
        if xrange is not None:
            plt.xlim(xrange)
        if yrange is not None:
            plt.ylim(yrange)
        # x and y scales
        if ylog:
            plt.yscale('log')
        if xlog:
            plt.xscale('log')
        plt.subplots_adjust(top=1.0)
        
        if export_figures and save_name is not None:
            plt.savefig(output_dir + save_name, bbox_inches="tight")

In [6]:
all_results["n_rects"].unique()

array([   100,   1000,   5000,  10000,  20000,  30000,  40000,  50000,
        60000,  70000,  80000,  90000, 100000], dtype=int64)

In [7]:
render_plot(xfeature="n_rects", yfeature="avg_dist", xlabel="Number of Hyperrectangles", ylabel="Explanation Distance", title=None, save_name="dist_vs_rects.png", loc="upper center", lhorz=True)

In [8]:
render_plot(xfeature="n_rects", yfeature="prep_time", xlabel="Number of Rectangles", ylabel="Initialization Time (sec)", title=None, save_name="prep_vs_rects.png", loc="upper center", lhorz=True)

In [9]:
render_plot(xfeature="n_rects", yfeature="sample_time", xlabel="Number of Rectangles", ylabel="Sample Explanation Time (sec)", title=None, save_name="explain_vs_nrects", loc="upper center", lhorz=True)